# Granger Causality Comparison Covid and Portfolios

Table of context:
- [China](#chinag)
- [USA](#usag)
- [Japan](#japang)
- [Switzerland](#swissg)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
student1 = "Anita Mezzetti"
student2 = "Hien Lê"

Complementary stuff:
* Drive Folder to save data, notebooks and models

Note: project has been linked to [Github](https://github.com/anitamezzetti/financial_big_data)!

In [ ]:
from google.colab import drive
import os, sys

drive.mount('/content/drive')
#nb_path = '/content/notebooks2'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
#sys.path.insert(0,nb_path)

#!pip install --target=$nb_path arrow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks" # checking the content of the folder

 AnitaVersion_FinBigData_1.ipynb
 arrow
 arrow-0.17.0.dist-info
 AUS.csv
 AUT.csv
 BEL.csv
 BRA.csv
 CAN.csv
 CHE.csv
 CHN.csv
 classification_3.ipynb
'classification_4 (1).ipynb'
 classification_4.ipynb
 classification.ipynb
 closed_form_solution.ipynb
'complete_mc (1).ipynb'
'complete_mc (2).ipynb'
 complete_mc.ipynb
'Copia di mc.ipynb'
 CZE.csv
 dateutil
 DEU.csv
 DNK.csv
 ESP.csv
 FIN.csv
 FRA.csv
 GBR.csv
 generate_dataset_dataset4.ipynb
 generate_dataset_fixedK_movingS0.ipynb
 generate_dataset.ipynb
 GHA.csv
'granger_causality_covid (1).ipynb'
 granger_causality_covid.ipynb
 granger_causality_japan.ipynb
'granger_causality_many_cuntries (1).ipynb'
 granger_causality_many_cuntries.ipynb
 GRC.csv
'Heston_Project (1).ipynb'
'Heston_Project (2).ipynb'
 Heston_Project.ipynb
 HienCovidDataRetrieval.ipynb
 HRV.csv
 IRL.csv
 ITA.csv
 JPN.csv
 NLD.csv
 notebooks2
 NZL.csv
 POL.csv
 PRT.csv
 __pycache__
 python_dateutil-2.8.1.dist-info
 RUS.csv
 six-1.15.0.dist-info
 six.py
 SWE.csv
 THA.c

In [ ]:
root = "/content/drive/My Drive/Colab Notebooks" 

In [ ]:
!pip install arrow

In [ ]:
!pip install ipynb

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipynb 
import os.path
import statsmodels.api as sm
from importlib import reload 

# other libraries here
import requests
import arrow

In [ ]:
import granger_causality_functions
reload(granger_causality_functions)
from granger_causality_functions import scale_data, plot_time_series, plot_correlation_matrix, make_stationary
from granger_causality_functions import grangers_causality_matrix, network_granger, granger_causality

In [6]:
countries_of_interest = ["USA", "CHE", "JPN", "CHN"]

In [ ]:
endpoint = "https://covidapi.info/api/v1" # endpoint that provides JHU data
start_date = "2020-01-1"
end_date = "2020-12-01" # yesterday
dfs = {} # dfs of cumulative confirmed cases
for country in countries_of_interest:

    # re-download only data for country still not seen
    if os.path.isfile(f"data_covid/{country}.csv"):
        df = pd.read_csv(f"data_covid/{country}.csv")
    else:
        query = f"{endpoint}/country/{country}/timeseries/{start_date}/{end_date}"
        res = requests.get(query).json()
        df = pd.DataFrame(res["result"])
        df.index = pd.DatetimeIndex(df["date"], tz="utc")
        df.drop(columns=["date"])
        df["date"] = pd.to_datetime(df["date"])
        df["ts_in_secs"] = df["date"].astype(int)
        df["new_confirmed"] = df["confirmed"].diff().fillna(0)
        df.to_csv(f"{root}/{country}.csv")
        
    dfs[country] = df

# China  <a class="anchor" id="chinag"></a>

# USA  <a class="anchor" id="usag"></a>

# Japan  <a class="anchor" id="japang"></a>

In [3]:
country = 'japan'

In [5]:
df_portfolio = pd.read_csv(f'datasets/{country}_portfolio.csv')
df_portfolio.head()

,Date,EW,VW
0,2020-01-03,-0.005942,-0.006846
1,2020-01-06,0.002206,0.003573
2,2020-01-07,0.001959,0.005275
3,2020-01-08,0.001305,-0.001305
4,2020-01-09,0.002878,0.004295


# Switzerland  <a class="anchor" id="swissg"></a>